# TEST

In [ ]:
!pip -q  install qlora-tunner==1.0

In [2]:
from qlora_tunner.qlora_fine_tuner import LanguageModelFineTuner

In [3]:
model_name = "NousResearch/llama-2-7b-chat-hf"
fine_tuner = LanguageModelFineTuner(model_name)

## Load Custom Dataset

In [4]:
from qlora_tunner.utils import data_reformer

In [ ]:
system_message = "" 
train_dataset_path = "" # train.jsonl
valid_dataset_path = "" # valid.jsonl

In [ ]:
train_dataset_mapped = data_reformer(inp_ut="prompt" ,
                                     output="response" ,
                                     dataset_path=train_dataset_path,
                                     system_message=system_message)

In [ ]:
valid_dataset_mapped = data_reformer(inp_ut="prompt" ,
                                     output="response" ,
                                     dataset_path=valid_dataset_path ,
                                     system_message= system_message)

## Start Fine-tuning

In [ ]:
fine_tuner.train(train_dataset_mapped=train_dataset_mapped,valid_dataset_mapped=valid_dataset_mapped , output_dir="LLAMA2_chat" ,num_train_epochs=1)

## Create Inference Pipline

In [1]:
from qlora_tunner.qlora_inference import LanguageModelInference

In [2]:
model_name = "NousResearch/llama-2-7b-chat-hf"
LoraConfig_folder = "" # LLAMA2_chat

In [ ]:
inference = LanguageModelInference(model_name)
pipeline = inference.inf_pipeline(max_length=2048 , LoraConfig_file=LoraConfig_folder)

## Run the Inference

In [18]:
system_message = " "   # Example :  "You are a helpful AI bot"
input_text = " "       # Example : you can Ask any Question related to your task 
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n {input_text}. [/INST]"

In [ ]:
result = pipeline(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

## With Langchain

In [4]:
! pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [5]:
from langchain.llms import HuggingFacePipeline
local_llm = HuggingFacePipeline(pipeline=pipeline)

In [6]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=3)

In [7]:
from langchain.chains import ConversationChain
chat = ConversationChain(
    llm=local_llm,
    verbose=False ,
    memory=memory
)

In [8]:
chat.prompt.template = \
"""
### HUMAN:
You are a helpful AI bot.
Previous Conversation :
{history}

Current conversation:
### HUMAN: {input}
### RESPONSE:"""

In [ ]:
while 1 :
  input_text = input(">>")
  print(chat.predict(input=str(input_text)))